## 4. Predictive Analysis 

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

# Set some specs for plotting
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16.0, 8.0)
mpl.style.use('ggplot')
plt.rcParams['font.family'] = 'IPAGothic'

# Read data from the data folder
race_df = pd.read_csv('data/race.csv', low_memory=False, index_col=0)
horse_df = pd.read_csv('data/horse.csv', low_memory=False, index_col=0)
individual_df = pd.read_csv('data/individual.csv', low_memory=False, index_col=0)
trainer_df = pd.read_csv('data/trainer.csv', low_memory=False, index_col=0)
jockey_df = pd.read_csv('data/jockey.csv', low_memory=False, index_col=0)
horse_race_df = pd.read_csv('data/horse_race.csv', low_memory=False, index_col=0)
horse_race_df['age_int'] = horse_race_df['sex_age'].apply(lambda x: re.search(r'\d+', x).group(0)).astype(int)

try:
    first_occur_df = pd.read_csv('data/first_occurence_race.csv', low_memory=False, index_col=0)
except FileNotFoundError:
    horse_race_sorted = horse_race_df.sort_values('horse_id')
    horse_id_set = set()
    first_occur_dict = {}
    for index, value in horse_race_sorted.iterrows():
        if value['horse_id'] not in horse_id_set:
            horse_id_set.add(value['horse_id'])
            first_occur_dict[index] = value
    first_occur_df = pd.DataFrame.from_dict(first_occur_dict, orient='index')

In [28]:
columns_to_drop = [
    # TODO: Update columns to drop when breeder id is availabl
    'run_date', 'race', 'title', 'horse', 'sex_age',
    'jockey', 'margin', 'trainer_x', 'trainer_y', 'owner_x', 'owner_y',
    'horse_name', 'date_of_birth', ''
]

,0,7,13,39,75,82,89,119,131,142,...,856873,856874,856875,856876,856877,856878,856879,856880,856881,856882
run_date,2000-01-09 10:00:00,2000-03-12 14:55:00,2000-01-09 10:00:00,2000-07-09 10:40:00,2001-07-08 13:50:00,2000-04-23 12:45:00,2000-03-04 10:50:00,2000-11-25 12:35:00,2000-07-09 12:45:00,2000-04-23 14:20:00,...,2018-01-28 10:40:00,2018-01-28 11:10:00,2018-01-28 10:40:00,2018-01-28 11:10:00,2018-01-28 10:40:00,2018-01-28 10:40:00,2018-01-28 10:10:00,2018-01-28 10:40:00,2018-01-28 10:40:00,2018-01-28 10:40:00
place,中山,中京,中山,函館,福島,東京,中山,東京,東京,東京,...,東京,東京,東京,東京,東京,東京,東京,東京,東京,東京
race,1R,10R,1R,3R,8R,6R,3R,6R,6R,9R,...,2R,3R,2R,3R,2R,2R,1R,2R,2R,2R
title,4歳未勝利,はなのき賞(500万下),4歳未勝利,4歳未勝利,4歳上500万下,4歳500万下,4歳未勝利,4歳上500万下,4歳未勝利,若鮎賞(500万下),...,3歳新馬,3歳未勝利,3歳新馬,3歳未勝利,3歳新馬,3歳新馬,3歳未勝利,3歳新馬,3歳新馬,3歳新馬
type,ダ,芝,ダ,ダ,ダ,ダ,ダ,ダ,芝,芝,...,ダ,ダ,ダ,ダ,ダ,ダ,ダ,ダ,ダ,ダ
track,右,左,右,右,右,左,右,左,左,左,...,左,左,左,左,左,左,左,左,左,左
distance,1200,1200,1200,1000,1000,1200,1200,2100,1600,1400,...,1400,1600,1400,1600,1400,1400,1300,1400,1400,1400
weather,曇,晴,曇,曇,晴,晴,曇,晴,晴,晴,...,曇,曇,曇,曇,曇,曇,曇,曇,曇,曇
condition,良,良,良,稍重,良,良,良,良,良,良,...,不良,不良,不良,不良,不良,不良,不良,不良,不良,不良
time,10:00,14:55,10:00,10:40,13:50,12:45,10:50,12:35,12:45,14:20,...,10:40,11:10,10:40,11:10,10:40,10:40,10:10,10:40,10:40,10:40


In [29]:
horse_df

,horse_id,horse_name,date_of_birth,trainer,owner,breeder,place_of_birth,transaction_price,prize_obtained,race_record,highlight_race,relatives,parents,status,gender,breed
0,1997103106,マルタカファイナル,1997年4月1日,新山広道 (愛知),高橋義和,橋本牧場,早来町,-,122万円 (地方),54戦1勝 [ 1-2-3-48 ],サラC18組,シャンハイジャパン 、 シャンハイセンプー,1983102584 1987100245,NaN,牡,栗毛
1,1995108282,エイシンワンダフル,1995年5月13日,湯浅三郎 (栗東),平井豊光,栄進牧場,浦河町,-,990万円 (中央),14戦0勝 [ 0-2-1-11 ],NaN,エイシンモモタロー 、 エイシンカガヤキ,000a00033a 1989108578,抹消,牝,鹿毛
2,1996110175,ビブラビブレ,1996年4月4日,高橋祥泰 (美浦),伊達秀和,Mrs. J. G. Jones Sr.,米,-,"2,491万円 (中央) /82万円 (地方)",20戦1勝 [ 1-6-4-9 ],00'障害3歳上未勝利,NaN,000a00199d 000a00aa53,抹消,牡,鹿毛
3,1996101001,リンクゴールド,1996年3月24日,湯窪幸雄 (栗東),阪口揚造,岩見牧場,新冠町,"1,575万円 (1997年 北海道6月セレクト市場)","2,127万円 (中央)",12戦1勝 [ 1-1-3-7 ],01'障害4歳上未勝利,トウキュウアビー 、 ジェットグローリ,1981107017 000a0062c7,抹消,牡,鹿毛
4,1997103190,ファイヤースキー,1997年6月2日,塚田隆男 (愛知),高草木茂男,芦谷清太,千歳,-,657万円 (地方),44戦3勝 [ 3-8-3-30 ],JRA認定,ロイヤルブリッジ 、 ボーウォーディン,1992110097 1988104728,NaN,牡,鹿毛
5,1996103385,ニホンピロタイラ,1996年5月15日,伊藤雄二 (栗東),小林百太郎,稲原牧場,平取町,-,"1,501万円 (中央)",18戦2勝 [ 2-0-1-15 ],00'4歳上500万下,ニホンピロファイブ 、 ニホンピロニール,000a001bed 000a006549,抹消,牝,青鹿毛
6,1997106681,ピアシルク,1997年3月25日,西橋豊治 (栗東),松屋隆士,ナカノファーム,門別町,-,304万円 (地方),19戦1勝 [ 1-0-0-18 ],フレッシュチャレンジ,テツリリー 、 ワンダーサファイア,000a00032c 1987101771,抹消,牝,芦毛
7,1995107973,ノットセルダム,1995年6月3日,木原一良 (栗東),名古屋友豊,ノーザンファーム,早来町,-,"1億1,809万円 (中央)",43戦5勝 [ 5-6-4-28 ],01'古都S(1600万下),サファイヤコースト 、 トーセンハピネス,000a00033a 000a0063ea,抹消,牡,鹿毛
8,1996100016,メイショウネルソン,1996年3月25日,武邦彦 (栗東),松本好雄,赤田牧場,浦河町,-,"3,455万円 (中央)",26戦3勝 [ 3-2-1-20 ],00'3歳上500万下,ユーワテイオー 、 セントラルマックス,000a00033a 1982102555,抹消,牡,黒鹿毛
9,1997105495,サカエワン,1997年5月2日,今津勝之 (地方),一柳榮二,八木常郎,新冠,-,282万円 (地方),7戦1勝 [ 1-1-0-5 ],ＪＲＡ認定,ハマノオイカゼ 、 サカエインパレスの1998,000a000365 1992107371,NaN,セ,鹿毛


## Some EDA to be integrated into descriptive analysis notebook 